In [ ]:
import osmnx as ox
import networkx as nx
import igraph as ig
import pandas as pd
import plotly.express as px


from roc_bike_growth.loader import POI_graph_from_polygon, bike_infra_from_polygon, carall_from_polygon
from roc_bike_growth.paper_gt import gt_with_existing_full
from roc_bike_growth.metrics import graph_resilience, graph_coverage, graph_cohesion

class BikeGraph:
    def __init__(self, prune_factor=0.1, route_factor=1):
        self.prune_factor = prune_factor
        self.route_factor = route_factor
        car_infra, bike_infra = self.get_data()
        self.bike_graph = self.merge_and_gt(car_infra, bike_infra)
        
    def get_data(self):
        rochester = ox.geocode_to_gdf('rochester, ny').geometry[0]
        bike_infra = bike_infra_from_polygon(rochester)
        car_infra = carall_from_polygon(rochester, add_pois=True)
        
        return car_infra, bike_infra
    
    def merge_and_gt(self, car_infra, bike_infra):
        return gt_with_existing_full(car_infra, bike_infra, self.route_factor, self.prune_factor)
    
    def display_metrics(self):
        res_density = graph_resilience(self.bike_graph, 'density')
        res_component = graph_resilience(self.bike_graph, 'largest_component')
        coverage = graph_coverage(self.bike_graph)
        cohesion = graph_cohesion(self.bike_graph, coverage)
        
        plot_metrics = 
        
        plot_data = pd.DataFrame(dict(
           theta = ['resilience (density)','resilience (largest_component)','coverage','cohesion'],
           r = [res_density, res_component, coverage, cohesion]
        ))
        fig = px.line_polar(plot_data, r='r', theta='theta', line_close=True)
        fig.show()
        